## CIA READINGROOM > CREST > SCIENTIFIC ABSTRACTS 

Scraping tests for metadata + pdf retrieval (pre-OCR tasks)<br>10.17.17

In [1]:
from bs4 import BeautifulSoup
import os
import requests
import urllib2
import pandas as pd

In [2]:
BASEURL = "https://www.cia.gov"
PAGINATE_PATH ="/library/readingroom/collection/scientific-abstracts?page="
RANGE = 1654 # pages, of 20 docs per page
TEST_RANGE = 5 # i.e. approx 100 docs

In [3]:
def retrieve_file(url, name):
    outpath = name + ".pdf"
    response = urllib2.urlopen(url)
    with open(outpath, 'w') as f:
        f.write(response.read())
        f.close()

In [4]:
def sciab_df(download_bool):
    
    df = pd.DataFrame(columns=['id','title','classification','publication_date'])
    idx = 0
    
    for i in xrange(TEST_RANGE):
        
        pagination_link = BASEURL + PAGINATE_PATH + str(i)
        print "Processing " + pagination_link
        pagination_page = requests.get(pagination_link)
        p_soup = BeautifulSoup(pagination_page.content, 'lxml')
        
        for doc_title in p_soup.find_all("h4", class_="field-content"):
            
            a = doc_title.select_one("a")
            link = a.get('href')
            TITLE = str(a.string)
    
            doc_page = requests.get(BASEURL + link)
            m_soup = BeautifulSoup(doc_page.content, 'lxml')
        
            PUB_DATE = str(m_soup.select_one(".field-name-field-pub-date").select_one("span").get('content'))
            ID = str(m_soup.select_one(".field-name-field-document-number").select_one(".field-item").string)
            CLASSIFICATION = str(m_soup.select_one(".field-name-field-original-classification").select_one(".field-item").string)
        
            if download_bool:
                PDF = m_soup.select_one(".file").select_one("a").get('href')
                retrieve_file(PDF, ID)
            
            df.loc[idx] = [ID, TITLE, CLASSIFICATION, PUB_DATE]
            idx+=1
            
    return df
                

In [5]:
sciab = sciab_df(False)

Processing https://www.cia.gov/library/readingroom/collection/scientific-abstracts?page=0
Processing https://www.cia.gov/library/readingroom/collection/scientific-abstracts?page=1
Processing https://www.cia.gov/library/readingroom/collection/scientific-abstracts?page=2
Processing https://www.cia.gov/library/readingroom/collection/scientific-abstracts?page=3
Processing https://www.cia.gov/library/readingroom/collection/scientific-abstracts?page=4


In [6]:
sciab

,id,title,classification,publication_date
0,CIA-RDP86-00513R001031010018-6,"""SCIENTIFIC ABSTRACT LVOVSKIY, P.G. - LYADOVA...",S,1967-12-31T01:01:01-05:00
1,CIA-RDP86-00513R000204310003-8,"""SCIENTIFIC ABSTRACT BELENKIY, M.N. - BELENKI...",S,1967-12-31T01:01:01-05:00
2,CIA-RDP86-00513R000204310001-0,"""SCIENTIFIC ABSTRACT BELENKIY, G. A. - BELENKI...",S,1967-12-31T01:01:01-05:00
3,CIA-RDP86-00513R000204310002-9,"""SCIENTIFIC ABSTRACT BELENKIY, L.I. - BELENKIY...",S,1967-12-31T01:01:01-05:00
4,CIA-RDP86-00513R000204310004-7,"""SCIENTIFIC ABSTRACT BELENKIY, N. - BELENKIY, ...",S,1967-12-31T01:01:01-05:00
5,CIA-RDP86-00513R000204310005-6,"""SCIENTIFIC ABSTRACT BELENKIY, V. I. - BELENKI...",S,1967-12-31T01:01:01-05:00
6,CIA-RDP86-00513R000204310006-5,"""SCIENTIFIC ABSTRACT BELENKIY, Y. Y. - BELENOV...",S,1967-12-31T01:01:01-05:00
7,CIA-RDP86-00513R000204310007-4,"""SCIENTIFIC ABSTRACT BELENKOV, V.P. - BELENOVS...",S,1967-12-31T01:01:01-05:00
8,CIA-RDP86-00513R000204310008-3,"""SCIENTIFIC ABSTRACT BELENSCHCHIKOV, L.D. - BE...",S,1967-12-31T01:01:01-05:00
9,CIA-RDP86-00513R000204310009-2,"""SCIENTIFIC ABSTRACT BELETSKAYA, I.P. - BELETS...",S,1967-12-31T01:01:01-05:00
